<div style="width: 100%; margin-left: auto; margin-right: auto;">
    <h2 style="text-align: center; margin-top: 1%; font-size:32px;">Avaliação de Desempenho - Atividade 6</h2>
</div>

**Aluno:** Igor Carvalho da Silva  
**Email:** igor.carvalho@icomp.ufam.edu.br  
**Matrícula:** 21951288  

---

## Introdução
Quando realizamos um experimento existem variáveis que afetam a variável resposta, chamados de fatores, como por exemplo tamanho da memória e carga do sistema. Ignorar esses fatores e não isolar os efeitos pode conduzir a experimentos ruins. Com isso, precisamos tentar reduzir a quantidade de fatores, escolhendo aqueles que têm impacto no desempenho utilizando o Projeto de Experimentos $2^{k}$ fatorial que irá ajudar a decidir qual fator é mais significativo.  

## Atividade
Implementar em Python o projeto de experimentos do tipo 2^k fatorial.  

### Dependências
Para reproduzir o experimento é necessário ter o `pip` instalado ou já possuir as bibliotecas instaladas.

In [1]:
!pip3 install numpy
!pip3 install pandas

     |████████████████████████████████| 12.2 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 499 kB 10.9 MB/s eta 0:00:01


### Imports

In [2]:
import itertools as it
import numpy as np
import pandas as pd
from functools import reduce

### a) O usuário deve escolher a quantidade de fatores entre k=2 e k=5;

In [29]:
k = int(input('Número de fatores (K, 2 <= K <= 5): '))

Número de fatores (K, 2 <= K <= 5): 3


### b) O usuário deve informar o maior valor (+1) e o menor valor (-1) de cada fator;

In [30]:
factors_values = []
for i in range(k):
    print(f'Para o fator {i+1} (k={i+1})')
    smaller = int(input('Menor valor: '))
    bigger = int(input('Maior valor: '))
    factors_values.append((smaller, bigger))

Para o fator 1 (k=1)
Menor valor: 1
Maior valor: 2
Para o fator 2 (k=2)
Menor valor: 4
Maior valor: 16
Para o fator 3 (k=3)
Menor valor: 1
Maior valor: 2


### c) O usuário deve informar quantas vezes cada execução piloto será repetida.

In [31]:
pilot_repeat = int(input('Quantas vezes cada execução piloto será repetida? '))

Quantas vezes cada execução piloto será repetida? 1


### d) A aplicação deve exibir a tabela de sinais; 

In [32]:
values = np.array(list(it.product([-1, 1], repeat=k))).T
factors = [chr(65+i) for i in range(k)]
signal_table = pd.DataFrame([1]*2**k,columns=["I"])
factors_combinations = reduce(lambda L, R: L + R, [list(it.combinations(factors, i)) for i in range(2, k + 1)])

for i in range(k):
    signal_table[factors[i]] = values[i]
    
for combination in factors_combinations:
    col = "".join(map(str,combination))
    signal_table[col] = 1
    for factor in combination:
        signal_table[col] *= signal_table[factor]
signal_table['y'] = ""
signal_table

,I,A,B,C,AB,AC,BC,ABC,y
0,1,-1,-1,-1,1,1,1,-1,
1,1,-1,-1,1,1,-1,-1,1,
2,1,-1,1,-1,-1,1,-1,1,
3,1,-1,1,1,-1,-1,1,-1,
4,1,1,-1,-1,-1,-1,1,1,
5,1,1,-1,1,-1,1,-1,-1,
6,1,1,1,-1,1,-1,-1,-1,
7,1,1,1,1,1,1,1,1,


### e) O usuário deve informar o(s) resultado(s) de uma execução piloto com aquela combinação;

In [33]:
results = []
print(f'''
    Digite os resultados das observações, caso houver, serparados por ",".
    Apenas os {pilot_repeat} primeiros resultados digitados serão considerados.
''', end='\n\n')

for f in range(k):
    print(f'Fator {f+1}  ', end='')
print('Resultados')

for j in range(2**k):
    signals = list(signal_table.iloc[j][1:k+1])
    values = [factors_values[f][0] if signal == -1 else factors_values[f][1] for f, signal in enumerate(signals)]
    print(str(values)[1:-1].replace(',',' '*7)+' '*7, end='')
    results.append([int(item) for item in input().split(',')[:pilot_repeat]])


    Digite os resultados das observações, caso houver, serparados por ",".
    Apenas os 1 primeiros resultados digitados serão considerados.


Fator 1  Fator 2  Fator 3  Resultados
1        4        1       14
1        4        2       22
1        16        1       10
1        16        2       16
2        4        1       18
2        4        2       20
2        16        1       22
2        16        2       24


In [34]:
signal_table['y'] = results
if pilot_repeat > 1:
    signal_table['y_mean'] = list(map(lambda x: np.mean(x),results))
signal_table

,I,A,B,C,AB,AC,BC,ABC,y
0,1,-1,-1,-1,1,1,1,-1,[14]
1,1,-1,-1,1,1,-1,-1,1,[22]
2,1,-1,1,-1,-1,1,-1,1,[10]
3,1,-1,1,1,-1,-1,1,-1,[16]
4,1,1,-1,-1,-1,-1,1,1,[18]
5,1,1,-1,1,-1,1,-1,-1,[20]
6,1,1,1,-1,1,-1,-1,-1,[22]
7,1,1,1,1,1,1,1,1,[24]


### f) A aplicação deve mostrar a fração de variação explicada por cada fator.

In [35]:
results = [np.mean(result) for result in results]

mean = np.mean(results)
sst = sum([(y - mean)**2 for y in results])
all_factors = factors + ["".join(map(str,combination)) for combination in factors_combinations]
totals = []

print('Frações de Variação por fator:')

for factor in all_factors:
    total_factor = np.dot(signal_table[factor].to_numpy(), np.array(results))
    totals.append(total_factor)
    frac = 2**k*((total_factor/2**k)**2)
    print(f'Fator {factor}: {frac}/{sst} ({round(100*frac/sst)}%)')

Frações de Variação por fator:
Fator A: 60.5/155.5 (39%)
Fator B: 0.5/155.5 (0%)
Fator C: 40.5/155.5 (26%)
Fator AB: 40.5/155.5 (26%)
Fator AC: 12.5/155.5 (8%)
Fator BC: 0.5/155.5 (0%)
Fator ABC: 0.5/155.5 (0%)


In [36]:
print(signal_table)
print('Total: ', totals)
print(f'Total/{2**k}: ', list(map(lambda t: t/2**k, totals)))

   I  A  B  C  AB  AC  BC  ABC     y
0  1 -1 -1 -1   1   1   1   -1  [14]
1  1 -1 -1  1   1  -1  -1    1  [22]
2  1 -1  1 -1  -1   1  -1    1  [10]
3  1 -1  1  1  -1  -1   1   -1  [16]
4  1  1 -1 -1  -1  -1   1    1  [18]
5  1  1 -1  1  -1   1  -1   -1  [20]
6  1  1  1 -1   1  -1  -1   -1  [22]
7  1  1  1  1   1   1   1    1  [24]
Total:  [22.0, -2.0, 18.0, 18.0, -10.0, -2.0, 2.0]
Total/8:  [2.75, -0.25, 2.25, 2.25, -1.25, -0.25, 0.25]


## Conclusão
No Projeto de Experimentos $2^{k}$, o número de pontos experimentais cresce numa progressão geométrica de razão 2. Ele é útil para investigar simultaneamente vários fatores para posteriormente afunilar a investigação naqueles que se mostrarem mais relevantes.